In [1]:
#import 相关库
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import tqdm
import sys
import os
import gc
import argparse
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 读取数据
train_power_forecast_history = pd.read_csv('./训练集/power_forecast_history.csv')
train_power = pd.read_csv('./训练集/power.csv')
train_stub_info = pd.read_csv('./训练集/stub_info.csv')

test_power_forecast_history = pd.read_csv('./测试集/power_forecast_history.csv')
test_stub_info = pd.read_csv('./测试集/stub_info.csv')

In [3]:
# 聚合数据
train_df = train_power_forecast_history.groupby(['id_encode','ds']).head(1)
del train_df['hour']

test_df = test_power_forecast_history.groupby(['id_encode','ds']).head(1)
del test_df['hour']

tmp_df = train_power.groupby(['id_encode','ds'])['power'].sum()
tmp_df.columns = ['id_encode','ds','power']

# 合并充电量数据
train_df = train_df.merge(tmp_df, on=['id_encode','ds'], how='left')

### 合并数据
train_df = train_df.merge(train_stub_info, on='id_encode', how='left')
test_df = test_df.merge(test_stub_info, on='id_encode', how='left')

In [4]:
train_df['flag'] = train_df['flag'].map({'A':0,'B':1})
test_df['flag'] = test_df['flag'].map({'A':0,'B':1})

In [6]:
!pip install h3

     ---------------------------------------- 0.0/848.6 kB ? eta -:--:--
     ---------------------------------------- 10.2/848.6 kB ? eta -:--:--
     ---------------------------------------- 10.2/848.6 kB ? eta -:--:--
     - ----------------------------------- 30.7/848.6 kB 330.3 kB/s eta 0:00:03
     - ----------------------------------- 30.7/848.6 kB 330.3 kB/s eta 0:00:03
     - ----------------------------------- 30.7/848.6 kB 330.3 kB/s eta 0:00:03
     - ----------------------------------- 41.0/848.6 kB 164.3 kB/s eta 0:00:05
     -- ---------------------------------- 61.4/848.6 kB 233.8 kB/s eta 0:00:04
     ---- -------------------------------- 92.2/848.6 kB 309.1 kB/s eta 0:00:03
     ---- ------------------------------- 112.6/848.6 kB 364.4 kB/s eta 0:00:03
     ---- ------------------------------- 112.6/848.6 kB 364.4 kB/s eta 0:00:03
     ------ ----------------------------- 143.4/848.6 kB 355.0 kB/s eta 0:00:02
     ------- ---------------------------- 174.1/848.6 kB 40

**地理位置**

In [7]:
import h3
train_df["h3"].head()
train_df['latitude'] = train_df['h3'].apply(lambda x: h3.h3_to_geo(x)[0])
train_df['longitude'] = train_df['h3'].apply(lambda x: h3.h3_to_geo(x)[1])
test_df['latitude'] = test_df['h3'].apply(lambda x: h3.h3_to_geo(x)[0])
test_df['longitude'] = test_df['h3'].apply(lambda x: h3.h3_to_geo(x)[1])

In [8]:
def get_time_feature(df, col):
    
    df_copy = df.copy()
    prefix = col + "_"
    df_copy['new_'+col] = df_copy[col].astype(str)
    
    col = 'new_'+col
    df_copy[col] = pd.to_datetime(df_copy[col], format='%Y%m%d')
    df_copy[prefix + 'year'] = df_copy[col].dt.year
    df_copy[prefix + 'month'] = df_copy[col].dt.month
    df_copy[prefix + 'day'] = df_copy[col].dt.day
    # df_copy[prefix + 'weekofyear'] = df_copy[col].dt.weekofyear
    df_copy[prefix + 'dayofweek'] = df_copy[col].dt.dayofweek
    df_copy[prefix + 'is_wknd'] = df_copy[col].dt.dayofweek // 6
    df_copy[prefix + 'quarter'] = df_copy[col].dt.quarter
    df_copy[prefix + 'is_month_start'] = df_copy[col].dt.is_month_start.astype(int)
    df_copy[prefix + 'is_month_end'] = df_copy[col].dt.is_month_end.astype(int)
    del df_copy[col]
    
    return df_copy   
    
train_df = get_time_feature(train_df, 'ds')
test_df = get_time_feature(test_df, 'ds')

#cols = [f for f in test_df.columns if f not in ['ds','power','h3']]
cols = [f for f in test_df.columns if f not in ['ds','power','h3']]

In [9]:
# 使用K折交叉验证训练和验证模型
def cv_model(clf, train_x, train_y, test_x, seed=2023):
    # 定义折数并初始化KFold
    folds = 8
    kf = KFold(n_splits=folds, shuffle=True, random_state=seed)
    
    # 初始化oof预测和测试集预测
    oof = np.zeros(train_x.shape[0])
    test_predict = np.zeros(test_x.shape[0])
    cv_scores = []

    # KFold交叉验证
    for i, (train_index, valid_index) in enumerate(kf.split(train_x, train_y)):
        print('************************************ {} ************************************'.format(str(i+1)))
        trn_x, trn_y, val_x, val_y = train_x.iloc[train_index], train_y[train_index], train_x.iloc[valid_index], train_y[valid_index]
        
        # 转换数据为lightgbm数据格式
        train_matrix = clf.Dataset(trn_x, label=trn_y)
        valid_matrix = clf.Dataset(val_x, label=val_y)

        # 定义lightgbm参数
        params = {
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': 'rmse',
            'min_child_weight': 5,
            'num_leaves': 2 ** 8,
            'lambda_l2': 10,
            'feature_fraction': 0.8,
            'bagging_fraction': 0.8,
            'bagging_freq': 4,
            'learning_rate': 0.01,
            'seed': 2023,
            'nthread' : 16,
            'verbose' : -1,
            'device':'gpu'
        }

        # 训练模型
        model = clf.train(params, train_matrix, 3000, valid_sets=[train_matrix, valid_matrix], categorical_feature=[])
        
        # 获取验证和测试集的预测值
        val_pred = model.predict(val_x, num_iteration=model.best_iteration)
        test_pred = model.predict(test_x, num_iteration=model.best_iteration)
        
        oof[valid_index] = val_pred
        test_predict += test_pred / kf.n_splits
        
        # 计算并打印当前折的分数
        score = np.sqrt(mean_squared_error(val_pred, val_y))
        cv_scores.append(score)
        print(cv_scores)
        
    return oof, test_predict

# 调用上面的函数进行模型训练和预测
lgb_oof, lgb_test = cv_model(lgb, train_df[cols], train_df['power'], test_df[cols])

************************************ 1 ************************************
[277.303184695154]
************************************ 2 ************************************
[277.303184695154, 254.06228014484296]
************************************ 3 ************************************
[277.303184695154, 254.06228014484296, 275.686616799416]
************************************ 4 ************************************
[277.303184695154, 254.06228014484296, 275.686616799416, 268.2966893805978]
************************************ 5 ************************************
[277.303184695154, 254.06228014484296, 275.686616799416, 268.2966893805978, 264.1673236183793]
************************************ 6 ************************************
[277.303184695154, 254.06228014484296, 275.686616799416, 268.2966893805978, 264.1673236183793, 260.81859498561766]
************************************ 7 ************************************
[277.303184695154, 254.06228014484296, 275.686616799416, 268.296689

In [10]:
#输出赛题提交格式的结果
test_df['power'] = lgb_test
test_df['power'] = test_df['power'].apply(lambda x: 0 if x<0 else x)
test_df[['id_encode','ds','power']].to_csv('result3.csv', index=False)